## KR260 AWGN DMA Stream


### Plot function for use in this notebook
The first code block below defines a function that we will use for plotting data throughout this notebook. Note that the function has a `n_samples` argument so that we can limit the number of samples to plot. Plotting more than a few thousand samples can be very slow and consume a lot of RAM.

### Requirements
Install the following:

```bash
pip install ipympl
pip install ipywidgets
pip install jupyter_bokeh
```

# Initilize

#### Bokeh Plot Function

In [167]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
import numpy as np
import math


def plot_to_notebook(in_signal,out_signal=None):
    t = np.linspace(0,len(in_signal),len(in_signal))
    output_notebook()
    p = figure(title = "Input & Output Signal")
    p.line(t,in_signal,legend_label="Input Signal",line_color="blue",line_width=2)
    if out_signal is not None:
        p.line(t,out_signal,legend_label="Output Signal",line_color="red",line_width=2)
    show(p)

#### Load Overlay

In [169]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import pynq.lib.dma
import time
import kria
import timestamp_regmap
import led_regmap
import awgn_regmap

# Load the overlay
overlay = Overlay('/home/root/jupyter_notebooks/kr260_awgn_dma_stream/kr260_awgn_dma_stream.bit')

# Assign blocks to short vars
dma          = overlay.axi_dma_0
led_module   = overlay.led_reg_0
timestamp    = overlay.Timestamp_0
awgn         = overlay.gng_top_0

##### Print out the register map for DMA and HLS Gain Blocks

In [ ]:
overlay.ip_dict

In [ ]:
dma.register_map

##### Display Time Stamp Register
The time stamp is burned into the FPGA during the build process

In [170]:
print("FPGA Build Timestamp:  " + kria.get_timestamp_str(timestamp))

FPGA Build Timestamp:  2025/2/11 2:3:40


##### Toggle USER_LED[1:0] on/off

In [91]:
for i in range(10):
    led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x1)
    time.sleep(0.25)
    led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x2)
    time.sleep(0.25)
led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x0)

##### Display AWGN Block Parameters

In [171]:
print("Noise Gain = " + str(awgn.read(awgn_regmap.RegMap.AWGN_NOISE_GAIN_ADDR)))
awgn.write(awgn_regmap.RegMap.AWGN_NOISE_GAIN_ADDR,4)
print("Noise Gain = " + str(awgn.read(awgn_regmap.RegMap.AWGN_NOISE_GAIN_ADDR)))
print("AWGN Enable = " + str(awgn.read(awgn_regmap.RegMap.AWGN_ENABLE_ADDR)))
awgn.write(awgn_regmap.RegMap.AWGN_ENABLE_ADDR,1)
print("AWGN Enable = " + str(awgn.read(awgn_regmap.RegMap.AWGN_ENABLE_ADDR)))

Noise Gain = 0
Noise Gain = 4
AWGN Enable = 0
AWGN Enable = 1


In [172]:
#awgn.write(gain_module.register_map.gain.address,4)
tBits, fBits = kria.get_format(awgn,awgn_regmap.RegMap.F_IN_ADDR)
print("F_IN : (" + str(tBits) + ", " + str(fBits) + ")")
tBits, fBits = kria.get_format(awgn,awgn_regmap.RegMap.F_OUT_ADDR)
print("F_OUT : (" + str(tBits) + ", " + str(fBits) + ")")
tBits, fBits = kria.get_format(awgn,awgn_regmap.RegMap.F_AWGN_ADDR)
print("F_AWGN : (" + str(tBits) + ", " + str(fBits) + ")")

type(fBits)

F_IN : (16, 14)
F_OUT : (16, 14)
F_AWGN : (16, 14)


int

### DMA Transfer the Sample buffer from ARM Processor to the AWGN FPGA Module, and plot

In [182]:
# Generate input samples
tBits, fBits = kria.get_format(awgn,awgn_regmap.RegMap.F_IN_ADDR)
#n = 1024
#samples = np.ones(n) * 2**(fBits)
#samples = samples.astype(np.int32)

# Create Sine Wave
T = 0.00002
# Sampling frequency
fs = 100e6
# Number of samples
n = int(T * fs)
# Time vector in seconds
t = np.linspace(0, T, n, endpoint=False)
# Samples of the signal
samples = np.sin(0.2e6*2*np.pi*t) 
# Convert samples to 32-bit integers
print('Number of samples: ',len(samples))
samples = samples * 2**(fBits)
samples = samples.astype(np.int32)
plot_to_notebook(samples)

# Read Gain Register
awgn.write(awgn_regmap.RegMap.AWGN_ENABLE_ADDR,1)

print("AWGN Gain           : " + str(awgn.read(awgn_regmap.RegMap.AWGN_NOISE_GAIN_ADDR)))
print("AWGN Enable         : " + str(awgn.read(awgn_regmap.RegMap.AWGN_ENABLE_ADDR)))
print("DMA Transfer Size   : " + str(len(samples)))

# Allocate buffers for the input and output signals
in_buffer = allocate(shape=(len(samples),), dtype=np.int32)
out_buffer = allocate(shape=(len(samples),), dtype=np.int32)

# Copy the samples to the in_buffer
np.copyto(in_buffer,samples)

# Trigger the DMA transfer and wait for the result
import time
start_time = time.time()
dma.sendchannel.transfer(in_buffer)
#print('Submit sending the transfer into in_buffer')
dma.recvchannel.transfer(out_buffer)
#print('Submit receiving the transfer from out_buffer')
dma.sendchannel.wait()
#print('Done Sending the transfer to in_buffer')
dma.recvchannel.wait()
#print('Done receiving the transfer from out_buffer')
stop_time = time.time()
hw_exec_time = stop_time-start_time

print('DMA Transfer Execution Time  :',hw_exec_time,' sec')

# Plot to the notebook
#plot_to_notebook(t,samples,2000,out_signal=out_buffer)

# Free the buffers
in_buffer.close()
out_buffer.close()
t = (out_buffer & 0xFFFF)
out_fp = []
for x in t:
    if(x>(2**(tBits-1))):
          x = x - (2**(tBits))
    out_fp.append(x/(2**fBits))
plot_to_notebook(samples/(2**fBits),out_fp)

Number of samples:  2000


Loading BokehJS ...

AWGN Gain           : 4
AWGN Enable         : 1
DMA Transfer Size   : 2000
DMA Transfer Execution Time  : 0.0022420883178710938  sec


Loading BokehJS ...

In [94]:
del in_buffer, out_buffer